In this example, the landmark_input and texture_input are additional inputs that represent facial landmark points and texture descriptors, respectively. The Concatenate() layer is used to concatenate these additional features with the output of the convolutional layers before the dense layers. The complete model is then compiled and trained using all inputs (X_train, landmark_train, and texture_train) with their corresponding labels.

In [ ]:
import cv2
import dlib
import numpy as np

# Load the face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('path/to/shape_predictor_68_face_landmarks.dat')

# Function to extract LBP features from an image
def extract_lbp_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = cv2.spatial_histogram(gray, [8,8], 256, True)
    return lbp.flatten()

# Function to extract facial landmarks and LBP features from an image
def extract_features(image_path):
    image = cv2.imread(image_path)
    # Detect faces in the image
    faces = detector(image)
    # Extract landmark and LBP features for each face
    landmarks = []
    textures = []
    for face in faces:
        # Extract landmark features
        shape = predictor(image, face)
        landmarks.append(np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)]))
        # Extract texture features
        texture = extract_lbp_features(cv2.resize(image[face.top():face.bottom(), face.left():face.right()], (256, 256)))
        textures.append(texture)
    return np.array(landmarks), np.array(textures)

# Example usage
landmark_train = []
texture_train = []
landmark_val = []
texture_val = []

# Iterate over training images
for i in range(len(train_image_paths)):
    landmarks, textures = extract_features(train_image_paths[i])
    landmark_train.append(landmarks)
    texture_train.append(textures)

# Iterate over validation images
for i in range(len(val_image_paths)):
    landmarks, textures = extract_features(val_image_paths[i])
    landmark_val.append(landmarks)
    texture_val.append(textures)

# Convert to numpy arrays
landmark_train = np.array(landmark_train)
texture_train = np.array(texture_train)
landmark_val = np.array(landmark_val)
texture_val = np.array(texture_val)


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming you have your landmark and texture features stored in arrays
landmark = ...
texture = ...

# Split the landmark and texture features into training and validation sets
landmark_train, landmark_val, texture_train, texture_val = train_test_split(landmark, texture, test_size=0.2, random_state=42)

In [ ]:
# Define the model architecture
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Add feature extraction layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))

# Define additional feature inputs
landmark_input = Input(shape=(68, 2), name='landmark_input')
texture_input = Input(shape=(256,), name='texture_input')

# Concatenate features with output of convolutional layers
conv_output = model.layers[-2].output
concat_layer = Concatenate()([conv_output, landmark_input, texture_input])

# Add dense layers
dense_layer = Dense(256, activation='relu')(concat_layer)
output_layer = Dense(num_classes, activation='softmax')(dense_layer)

# Define the complete model
complete_model = Model(inputs=[model.input, landmark_input, texture_input], outputs=output_layer)

# Compile the model
complete_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Train the model with all inputs
history = complete_model.fit([X_train, landmark_train, texture_train], y_train, epochs=10, batch_size=32, validation_data=([X_val, landmark_val, texture_val], y_val))


In [ ]:
## Trying adding features

In [ ]:
import cv2
from skimage.feature import hog, local_binary_pattern
from skimage.filters import gabor_kernel
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# load the landmark_train, texture_train, landmark_val, and texture_val datasets

# extract HOG features
def extract_hog_features(images):
    features = []
    for img in images:
        img = cv2.resize(img, (64, 64))
        fd = hog(img, orientations=9, pixels_per_cell=(8, 8),
                 cells_per_block=(2, 2), visualize=False, multichannel=True)
        features.append(fd)
    return features

train_hog = extract_hog_features(landmark_train)
val_hog = extract_hog_features(landmark_val)

# extract LBP features
def extract_lbp_features(images):
    features = []
    for img in images:
        img = cv2.resize(img, (64, 64))
        lbp = local_binary_pattern(img, 8, 1, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        features.append(hist)
    return features

train_lbp = extract_lbp_features(texture_train)
val_lbp = extract_lbp_features(texture_val)

# extract Gabor features
def extract_gabor_features(images):
    features = []
    for img in images:
        img = cv2.resize(img, (64, 64))
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        feats = []
        for k in range(3):
            kernel = np.real(gabor_kernel(0.4, theta=k*np.pi/3, sigma_x=1, sigma_y=1))
            filtered = cv2.filter2D(gray_img, cv2.CV_8UC3, kernel)
            feats.append(filtered)
        feats = np.array(feats)
        features.append(feats.flatten())
    return features

train_gabor = extract_gabor_features(texture_train)
val_gabor = extract_gabor_features(texture_val)

# concatenate all features
train_features = np.concatenate([train_hog, train_lbp, train_gabor], axis=1)
val_features = np.concatenate([val_hog, val_lbp, val_gabor], axis=1)

# define the model architecture
inputs = Input(shape=(train_features.shape[1],))
x = Dense(128, activation='relu')(inputs)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

# compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999), metrics=['accuracy'])

# train the model
history = model.fit(train_features, train_Y, batch_size=32, epochs=50, validation_data=(val_features, val_Y))
